In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tqdm import trange
import pickle

In [2]:
def CosineSimilarity(Text_list):
    array = cosine_similarity(Text_list[0:1], Text_list)
    array = array[0]
    avg_similarity = np.average(array[array != array[0]])
    return avg_similarity

In [3]:
Naver_user = pd.read_csv('A_V_Adj_C_df_naver_post_table.csv',encoding='utf-8',sep='\t')
Naver_page = pd.read_csv('A_V_Adj_C_df_naver_page_table.csv',encoding='utf-8',sep='\t')

In [4]:
origin_Naver_user = Naver_user

In [5]:
Naver_user = Naver_user.dropna(subset=['Embedding_text'])

In [6]:
# 사진 포함 33
Naver_user['Embedding_text'] = Naver_user['Embedding_text'].apply(lambda x : " ".join(x.split(' ')[:-33]) if len(x.split(' ')) > 33 else x )

C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [7]:
dbdbdeep_user = pd.read_csv('A_V_Adj_C_df_dbdbdeep_post_table.csv',encoding='utf-8',sep='\t')
dbdbdeep_page = pd.read_csv('A_V_Adj_C_df_dbdbdeep_page_table.csv',encoding='utf-8',sep='\t')

In [8]:
origin_dbdbdeep_user = dbdbdeep_user

In [9]:
dbdbdeep_user = dbdbdeep_user.dropna(subset=['Embedding_text'])

In [10]:
Naver_page = Naver_page.dropna(subset=['Embedding_text'])
dbdbdeep_page = dbdbdeep_page.dropna(subset=['Embedding_text']) 

# TFIDF

## Model_load

In [11]:
v_load = pickle.load(open("Text_features_100.pkl", "rb"))

In [29]:
v_load.transform([Naver_user.iloc[0]['Embedding_text']]).toarray().flatten()

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.43610134, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.59167331, 0.        ,
       0.        , 0.5029914 , 0.        , 0.        , 0.26269378,
       0.37111985, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [39]:
def tfidf_vectorizer(Text):
    try:
        return v_load.transform([Text]).toarray().flatten()
    except:
        return ''

In [40]:
Text_similarity = []
for i in trange(len(Naver_user)):
    try:
        User_id = Naver_user.iloc[i]['User_id']
        Text = tfidf_vectorizer(Naver_user.iloc[i]['Embedding_text'])
        Text_list = Naver_page[Naver_page['User_id'] == User_id]['Embedding_text'].tolist()
        Text_list = list(map(lambda x : tfidf_vectorizer(x),Text_list))
        if len(Text_list) > 4:
            Text_list = Text_list[:-3]
        Text_list.insert(0,Text)        
        Text_Similarity_result = CosineSimilarity(Text_list)
        Text_similarity.append(Text_Similarity_result)
    except:
        Text_similarity.append('')
        print(i)

  1%|▋                                                                              | 79/8865 [00:00<00:46, 190.88it/s]C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\numpy\lib\function_base.py:356: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|█████████████████████████████████████████████████████████████████████████████| 8865/8865 [00:43<00:00, 203.90it/s]


In [41]:
Naver_user['Text_similarity'] = Text_similarity

C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [42]:
origin_Naver_user.shape

(9942, 19)

In [43]:
similarity = Naver_user[['Post_id','Text_similarity']]

In [44]:
revised_df = pd.merge(origin_Naver_user,similarity,how='outer')

In [45]:
naver_similarity = Naver_user['Text_similarity'].dropna()

In [69]:
naver_average = naver_similarity.sum()/len(naver_similarity)

In [70]:
((naver_similarity-naver_average)**2).sum()/len(naver_similarity)

0.018016458890298778

In [47]:
len(naver_similarity)

8441

In [48]:
revised_df.to_csv('[Revised]TFIDF_Final_Naver_post_table.csv',sep='\t',encoding='utf-8',index=False)

# dbdbdeep

In [49]:
dbdbdeep_Text_similarity = []
for i in trange(len(dbdbdeep_user)):
    try:
        User_id = dbdbdeep_user.iloc[i]['User_id']
        Text = tfidf_vectorizer(dbdbdeep_user.iloc[i]['Embedding_text'])
        Text_list = dbdbdeep_page[dbdbdeep_page['User_id'] == User_id]['Embedding_text'].tolist()
        Text_list = list(map(lambda x : tfidf_vectorizer(x),Text_list))
        if len(Text_list) > 4:
            Text_list = Text_list[:-3]
        Text_list.insert(0,Text)        
        Text_Similarity_result = CosineSimilarity(Text_list)
        dbdbdeep_Text_similarity.append(Text_Similarity_result)
    except:
        dbdbdeep_Text_similarity.append('')

  0%|▏                                                                              | 19/6208 [00:00<00:33, 182.75it/s]C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\numpy\lib\function_base.py:356: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|█████████████████████████████████████████████████████████████████████████████| 6208/6208 [00:31<00:00, 196.89it/s]


In [50]:
dbdbdeep_user['Text_similarity'] = dbdbdeep_Text_similarity

C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [51]:
origin_dbdbdeep_user.shape

(6362, 19)

In [52]:
dbdbdeep_similarity = dbdbdeep_user[['Post_id','Text_similarity']]

In [53]:
revised_dbdbdeep_df = pd.merge(origin_dbdbdeep_user,dbdbdeep_similarity,how='outer')

In [54]:
dbdbdeep_similarity = dbdbdeep_user['Text_similarity'].dropna()

In [60]:
average = dbdbdeep_similarity.sum()/len(dbdbdeep_similarity)

In [71]:
((dbdbdeep_similarity-average)**2).sum()/len(dbdbdeep_similarity)

0.008398125508382254

In [72]:
revised_dbdbdeep_df.to_csv('[Revised]TFIDF_Final_dbdbdeep_post_table.csv',sep='\t',encoding='utf-8',index=False)

# T-test (Similarity)

In [73]:
from scipy import stats
# If you have two independent samples but you do not know that they have equal variance, you can use Welch's t-test
# equal_var=False
similarity = stats.ttest_ind(naver_similarity,dbdbdeep_similarity, equal_var=False)

In [74]:
similarity

Ttest_indResult(statistic=-12.381594515474958, pvalue=4.951134097254685e-35)

# Naver

## Date

In [ ]:
user_id = pd.read_csv('[Final]Naver_user_table.csv',sep='\t',encoding='utf-8')

In [ ]:
User_list = user_id['User_id'].tolist()

### Year,month,date

In [ ]:
Naver_page = Naver_page[~Naver_page['Date'].str.contains('전')]

In [ ]:
Naver_page = Naver_page[Naver_page['Date'] != ' ']

In [ ]:
from datetime import date
Time_interval_average = []
Time_interval_std = []
post_average_time = []
post_std = []
error_list = []
for i in trange(len(User_list)):
    Date_list = Naver_page[Naver_page['User_id'] == User_list[i]]['Date'].tolist()
    Year = []
    Month = []
    Date = []
    Hour = []
    if len(Date_list) == 1:
        Time_interval_average.append('')
        Time_interval_std.append('')
        post_average_time.append('')
        post_std.append('')
        error_list.append(User_list[i])
    else:
        if len(Date_list) > 4:
            Date_list = Date_list[:-3]
        for date in Date_list:
            raw_date = date.split('.')
            Year.append(int(raw_date[0]))
            Month.append(int(raw_date[1]))
            Date.append(int(raw_date[2]))
            Hour.append(int(raw_date[3].split(':')[0]))

            difference = []
            for j in range(0,len(Year)):
                if j != len(Year)-1:
                    from datetime import date
                    l_date = date(Year[j],Month[j],Date[j])
                    f_date = date(Year[j+1],Month[j+1],Date[j+1])
                    delta = l_date - f_date
                    difference.append(delta.days)
       
        Time_interval_average.append(np.average(np.array(difference)))
        Time_interval_std.append((np.std(np.array(difference))))
        post_average_time.append(np.average(np.array(Hour)))
        post_std.append(np.std(np.array(Hour)))

In [ ]:
user_id['Post_interval_average'] = Time_interval_average
user_id['Post_interval_std'] = Time_interval_std
user_id['Post_average_time'] = post_average_time
user_id['Post_std'] = post_std

In [ ]:
user_id.to_csv('[Revised]Final_Naver_user_table.csv',sep='\t',encoding='utf-8',index=False)

In [ ]:
test = user_id[user_id['Post_interval_average'] != '']['Post_interval_average']

In [ ]:
test.sum()/len(test)

In [ ]:
test = user_id[user_id['Post_interval_std'] != '']['Post_interval_std']

In [ ]:
test.sum()/len(test)

In [ ]:
test = user_id[user_id['Post_average_time'] != '']['Post_average_time']

In [ ]:
test.sum()/len(test)

In [ ]:
test = user_id[user_id['Post_std'] != '']['Post_std']

In [ ]:
test.sum()/len(test)

# DBDBDEEP

## Date

In [ ]:
user_id = pd.read_csv('[Final]dbdbdeep_user_table.csv',sep='\t',encoding='utf-8')

In [ ]:
User_list = user_id['User_id'].tolist()

### Year,month,date

In [ ]:
dbdbdeep_page = dbdbdeep_page[~dbdbdeep_page['Date'].str.contains('전')]

In [ ]:
dbdbdeep_page = dbdbdeep_page[dbdbdeep_page['Date'] != ' ']

In [ ]:
from datetime import date
Time_interval_average = []
Time_interval_std = []
post_average_time = []
post_std = []
error_list = []
for i in trange(len(User_list)):
    Date_list = dbdbdeep_page[dbdbdeep_page['User_id'] == User_list[i]]['Date'].tolist()
    Year = []
    Month = []
    Date = []
    Hour = []
    if len(Date_list) == 1:
        Time_interval_average.append('')
        Time_interval_std.append('')
        post_average_time.append('')
        post_std.append('')
        error_list.append(User_list[i])
    else:
        if len(Date_list) > 4:
            Date_list = Date_list[:-3]
        for date in Date_list:
            raw_date = date.split('.')
            Year.append(int(raw_date[0]))
            Month.append(int(raw_date[1]))
            Date.append(int(raw_date[2]))
            Hour.append(int(raw_date[3].split(':')[0]))

            difference = []
            for j in range(0,len(Year)):
                if j != len(Year)-1:
                    from datetime import date
                    l_date = date(Year[j],Month[j],Date[j])
                    f_date = date(Year[j+1],Month[j+1],Date[j+1])
                    delta = l_date - f_date
                    difference.append(delta.days)
       
        Time_interval_average.append(np.average(np.array(difference)))
        Time_interval_std.append((np.std(np.array(difference))))
        post_average_time.append(np.average(np.array(Hour)))
        post_std.append(np.std(np.array(Hour)))

In [ ]:
user_id['Post_interval_average'] = Time_interval_average
user_id['Post_interval_std'] = Time_interval_std
user_id['Post_average_time'] = post_average_time
user_id['Post_std'] = post_std

In [ ]:
user_id.to_csv('[Revised]Final_dbdbdeep_user_table.csv',sep='\t',encoding='utf-8',index=False)

In [ ]:
test = user_id[user_id['Post_interval_average'] != '']['Post_interval_average']

In [ ]:
test.sum()/len(test)

In [ ]:
test = user_id[user_id['Post_interval_std'] != '']['Post_interval_std']

In [ ]:
test.sum()/len(test)

In [ ]:
test = user_id[user_id['Post_average_time'] != '']['Post_average_time']

In [ ]:
test.sum()/len(test)

In [ ]:
test = user_id[user_id['Post_std'] != '']['Post_std']

In [ ]:
test.sum()/len(test)